In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-08-19 00:26:38--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0s      

2021-08-19 00:26:38 (57.6 MB/s) - ‘spacex_launch_dash.csv’ saved [3033/3033]



In [2]:
!pip3 install dash

     |████████████████████████████████| 1.1 MB 23.2 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 7.3 MB/s  eta 0:00:01
     |████████████████████████████████| 3.7 MB 45.3 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 1.8 MB 40.6 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 30.9 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 45.7 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-1.21.0-py3-none-any.whl size=1100478 sha256=70513a25899da252fbb72cb28f81e5a7a2c96512a15f640db73eaa056f38a8ef
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/56/e4/21/8a1190a7c31f3aa3f8834b05126ded8bf105b8fd11fad1806e
  Created wheel for dash-core-components: filename=dash_core_components-1.17.1-py3-none-any.whl size=3738983 sha256=ebb9dd890ff173be60afe1abbe9a5dcb6bbb12560b7755d3c29dacd1a124a712
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/08/ae/6a/0f4a0ee8fa216e

In [3]:
!pip3 install jupyter_dash

In [4]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
from dash import no_update
from jupyter_dash import JupyterDash

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

site_dropdown=set(spacex_df['Launch Site'])
site_dropdown.add('All Sites')

# Create a dash application
app = JupyterDash(__name__)



# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                html.Br(),
                                dcc.Dropdown(id='site_dropdown',
                                                    value='All Sites',
                                                    options=
                                                    [{'value': x, 'label':x} for x in site_dropdown],
                                                    placeholder='Select a Launch Site here',
                                                    searchable=True),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload_slider',
                                               min=0,
                                               max=10000,
                                               step=1000,
                                               value=[min_payload,max_payload],
                                               marks={
                                                   0:'0',
                                                   2500:'2500',
                                                   5000:'5000',
                                                   7500:'7500',
                                                   10000:'10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site_dropdown', component_property='value'))
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
# add callback decorator
#@app.callback( Output(component_id='success-payload-scatter-chart', component_property='figure'),
               #Input(component_id='payload_slider', component_property='value'))    
def pie_chart(site_dropdown):
    if site_dropdown == "All Sites":
        a = pd.crosstab(spacex_df['Launch Site'],spacex_df['class'])
        values=list(a.loc[:,1])
        labels=a.index
        fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
        return fig
    else:
        labels = [0,1]
        values = pd.crosstab(spacex_df['Launch Site'],spacex_df['class']).apply(lambda r: r/r.sum(), axis=1)
        values=[values.loc[site_dropdown,0],values.loc[site_dropdown,1]]
        fig = go.Figure(data=[go.Pie(labels=labels, values=values)]) 
        return fig
    
@app.callback( 
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site_dropdown', component_property='value'),
     Input(component_id='payload_slider', component_property='value')]) 

def scart_chart(site_dropdown,payload):
    min_payload, max_payload = payload
    if site_dropdown == "All Sites":
        df = spacex_df[(spacex_df['Payload Mass (kg)']>=min_payload) & (spacex_df['Payload Mass (kg)']<= max_payload)]
        fig = px.scatter(df, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig
    else:
        individual = spacex_df[spacex_df['Launch Site']==site_dropdown]
        individual = individual[(individual['Payload Mass (kg)']>=min_payload) & (individual['Payload Mass (kg)']<= max_payload)]
        fig = px.scatter(individual, x="Payload Mass (kg)", y="class", color="Booster Version Category")
        return fig


# Run the app
if __name__ == '__main__':
    app.run_server(mode="inline", host="localhost")